In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
04_eval_test.py
----------------
Purpose:
    Evaluate the saved Logistic Regression model on the HELD-OUT TEST split.
    Save: classification report (JSON) and confusion matrix (PNG) to OUTPUT/.

Assumptions:
    - ../DATA/splits/test.csv
    - ../OUTPUT/logreg.joblib
    - Columns: "Label", "SMS_Message"; no schema checks here.

Outputs:
    - ../OUTPUT/test_report.json
    - ../OUTPUT/confusion_matrix.png
"""
import json
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

splits_dir = Path("../DATA/splits")
out_dir = Path("../OUTPUT")
out_dir.mkdir(parents=True, exist_ok=True)

model_path = out_dir / "logreg.joblib"
test = pd.read_csv(splits_dir / "test.csv")

X_test = test["SMS_Message"].tolist()
y_test = (test["Label"].str.lower() == "spam").astype(int).values

model = joblib.load(model_path)

# Predict
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Metrics
report = classification_report(y_test, y_pred, target_names=["ham", "spam"], output_dict=True)
report["roc_auc"] = float(roc_auc_score(y_test, y_prob))

with open(out_dir / "test_report.json", "w") as f:
    json.dump(report, f, indent=2)

# Confusion matrix PNG
cm = confusion_matrix(y_test, y_pred)
fig = plt.figure()
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix (Test)")
plt.xticks([0, 1], ["ham", "spam"])
plt.yticks([0, 1], ["ham", "spam"])
for (i, j), v in np.ndenumerate(cm):
    plt.text(j, i, str(v), ha="center", va="center")
plt.xlabel("Predicted")
plt.ylabel("True")
fig.savefig(out_dir / "confusion_matrix.png", bbox_inches="tight")
plt.close(fig)

print("Wrote:", out_dir / "test_report.json")
print("Wrote:", out_dir / "confusion_matrix.png")


Wrote: ../OUTPUT/test_report.json
Wrote: ../OUTPUT/confusion_matrix.png
